# INIT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# MODULES PATH
import sys
sys.path.append('/content/drive/MyDrive/TFM_Retail_Repo/03_Clustering')

from modules import kmeans_module as km_m
from modules import preprocessing_module as pm


## Importacion datos

In [ ]:
from modules.get_data import get_data

In [ ]:
df_cal = get_data('exp_calendar.csv')
df_items = get_data('exp_items.csv')
df_shops = get_data('exp_shops.csv')
# df_ws = get_data('exp_weekly_sales.csv')
df_ws = get_data('daily_sales_with_events.csv') # Vamos a probar con diarias -> filtramos por años para que no pete la creacion de caracteristicas

In [ ]:
# Seleccionar solo años 2014 a 2016 de df_ws
df_ws = df_ws[(df_ws['year'] >= 2014) & (df_ws['year'] <= 2016)]

# Cambiamos NaN de event por None
df_ws['event'] = df_ws['event'].fillna('None')

display(df_ws.shape)

___
___

# CREACION FEATS. GENERALES

In [ ]:
!pip install featuretools # install the missing module

In [ ]:
from modules import feature_creation_module as fcm

In [ ]:
df_ws = df_ws.drop(columns=['event']) # no nos sirve ahora

In [ ]:
# Creacion caracteristicas globales para tiendas (luego iremos filtrando)
fc_config = {
    'target_df': 'shops',  # El DataFrame que contiene información sobre las tiendas
    'agg_primitives': [
        'min',               # Mínimo
        'max',               # Máximo
        'mean',              # Promedio
        'std',               # Desviación estándar
        'sum',               # Suma
        'count'              # Totales
    ],
    'trans_primitives': [
        # Ya tenemos el month en este df_ws
        # 'month',  # Extrae el mes para observar patrones estacionales a nivel semanal
        'is_weekend'  # Indica si la semana incluye fines de semana
    ],
    'max_depth': 2  # Profundidad máxima de las características generadas
}

feature_matrix, feature_defs = fcm.create_features(df_items, df_shops, df_ws, fc_config)

In [ ]:
feature_matrix.head()

## Descripciones

In [ ]:
# Revision definiciones caracteristicas creadas
features_descriptions = fcm.get_features_descriptions(feature_defs)

display(type(features_descriptions))

display(features_descriptions)

# **TIENDAS: PERFORMANCE VENTAS GENERAL**  
  
Esta configuración se centra en la evaluación general del rendimiento de ventas en cada tienda.


___
___

## Selección caracteristicas

In [ ]:
# # V3
# # substrings = ['raw_earn'] # Nos quedamos solo con las que afectan al precio de venta
# # substrings = ['units', 'raw_earn']

# feat_selected = fcm.select_features(features_descriptions, substrings)

# print(f'Length Feats Selected: {len(feat_selected)}')
# # display(feat_selected)

# # feature_matrix_selected = feature_matrix[feat_selected].copy()

# V4 -> Probamos solo con las caracteristicas de ventas
substrings = ['units', 'raw_earn']
feat_selected = fcm.select_features(features_descriptions, substrings)
feature_matrix_selected = feature_matrix[feat_selected].copy()
print(f'Length Feats Selected: {feature_matrix_selected.shape}')

# display(feature_matrix_selected.head(5))

# Aplicar filtros a las caracteristicas creadas
# Remove low information, highly correlated, highly null and single value features
# filtered_feature_matrix = fcm.filter_feature_matrix(feature_matrix_selected)
# V4 -> No filter
filtered_feature_matrix = feature_matrix_selected.copy()

# Renombrar características si es necesario # Evitamos este paso de momento
# feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, price_range_features)
feature_matrix_renamed = filtered_feature_matrix.copy()

display(feature_matrix_renamed.head(5))

## Preprocesamiento datos

In [ ]:
scaled_df = pm.preprocess_features(feature_matrix_renamed)

scaled_df.head()

## Kmeans

Elbow & Silhoutee

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 9)

In [ ]:
# optimal_k = 6 # valor mas alto de silhouette
optimal_k = 3 # codo + silueta

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

Analisis

In [ ]:
# Vemos varianza acumulada en componentes principales
variance_ratio = km_m.view_pca_variance(scaled_df)

acc_var = km_m.calculate_accumulated_variance(variance_ratio)

display(acc_var)

Importancia caracteristicas

In [ ]:
# Importancia de caracteristicas
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([scaled_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

IMPORTANCE_THRES=0.05
importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=IMPORTANCE_THRES)

# Seleccionamos solo las caracteristicas que tienen importancia por encima o igual del umbral
selected_features = importance_df[importance_df['Importance'] >= IMPORTANCE_THRES]

# Creamos una lista con las features más importantes usando la columna feature de importance_df
most_important_features = selected_features['Feature'].tolist()

# display(importance_df.head(10))

## Dim redux

In [ ]:
pca_df, importance_df = km_m.redu# Ejecutar PCA y clustering
n_components = 5 # Segun visto en varianza acumulada -> ???

# Obtenemos scaled df con componentes y clusters
df_with_pca, loadings_df = km_m.pca_and_cluster(scaled_df, n_clusters=optimal_k, n_components=n_components)

# # Mostrar el DataFrame con PCA y clustering
# display(df_with_pca.head(5))

# Graficar la cantidad de items por cluster
show_clusters = km_m.plot_cluster_sizes(df_with_pca)x_dimensions_pca_and_cluster(preprocessed_df, n_clusters=optimal_k, n_components=1)

## Descripcion de clusters

In [ ]:
# Visualizacion clusters
cols = n_components + 1 # -> Seleccionamos solo las columnas de PC + Cluster
df_pairplot = df_with_pca.iloc[:, -(cols):]

df_pairplot['Cluster'] = df_with_pca['Cluster']
sns.pairplot(df_pairplot[0:], hue='Cluster', palette='viridis')

In [ ]:
# T SNE visualizacion de clusters
km_m.tsne_visualization(df_pairplot, n_components=2, perplexity=30)

In [ ]:
# Seleccionamos las features según la lista most_important_features obtenida con random forest
importance_df_filt = importance_df[importance_df['Feature'].isin(most_important_features)]

# Crear y mostrar descripciones de los clusters
description_df = km_m.create_cluster_descriptions(df_with_pca, importance_df_filt)
print('###########################################################')
print('Descripcion de los clusters')
print('###########################################################')
# Ordenamos columnas de description_df usando sort
description_df = description_df[sorted(description_df.columns)]
display(description_df)

In [ ]:
# Añadimos clusters a df_items
df_shops = df_shops.reset_index(drop=True) # Recuperamos indices originales

df_shops_with_clusters = df_shops.merge(df_with_pca[['Cluster']], left_index=True, right_index=True)

df_shops_with_clusters.head()

In [ ]:
# Crear dataframe con las descripciones de los clusters
cluster_summary = {
    0: {
        'cluster_name': 'Tiendas con bajo rendimiento',
        'description': 'Tiendas que muestran consistentemente un bajo rendimiento en ingresos y ventas unitarias. Las métricas indican pocas ventas y baja variabilidad, lo que sugiere una base de clientes pequeña o falta de promociones efectivas.'
    },
    1: {
        'cluster_name': 'Tiendas con rendimiento promedio',
        'description': 'Tiendas con un rendimiento moderado en ingresos y ventas unitarias. Las tiendas en este clúster tienen una ligera variabilidad en sus métricas, lo que indica un desempeño estable pero con margen de mejora.'
    },
    2: {
        'cluster_name': 'Tiendas de alto rendimiento',
        'description': 'Tiendas con un rendimiento superior en ingresos y ventas unitarias. Estas tiendas destacan significativamente en sus métricas, con ingresos y ventas mucho mayores al promedio, posiblemente debido a una gran base de clientes o promociones efectivas.'
    }
}


cluster_summary_df = pd.DataFrame.from_dict(cluster_summary, orient='index')

# Asumiendo que tienes un dataframe `items_df` que tiene una columna 'cluster' con los números de cluster
# Realiza un merge con el resumen de clusters para añadir los nombres y descripciones al dataframe original
shops_with_cluster_info = df_shops_with_clusters.merge(cluster_summary_df, left_on='Cluster', right_index=True, how='left')

shops_with_cluster_info = shops_with_cluster_info.rename(columns={'Cluster':'cluster'})

# Resultado: `shops_with_cluster_info` tendrá las columnas adicionales 'cluster_name' y 'description'
display(shops_with_cluster_info.head())

In [ ]:
km_m.plot_clusters_with_name(shops_with_cluster_info)

## Guardar CSV

In [ ]:
# Guardamos df como csv
from google.colab import files

file_name = 'shops_global_sales_performance.csv'

# # Exportar DataFrame a CSV (reemplaza 'df' por el nombre de tu DataFrame)
# shops_with_cluster_info.to_csv(file_name, index=False)

# # Descargar el archivo CSV
# files.download(file_name)

# **MIX DE PRODUCTOS: Segun cluster performance ventas global**  
  
Esta configuración se centra en entender la variedad y popularidad de los productos en diferentes tiendas.
___
___

## Importacion de datos

In [ ]:
# Cargamos df de items con segmentacion segun ventas diarias de 2014 a 2016 de estos
df_items_sp_clusters = get_data('_clusters/items_clusters_raw_earn.csv') # Ya solo trabajamos con ventas diaris

In [ ]:
display(df_items_sp_clusters.head())
display(df_shops.head())
display(df_ws.head())

In [ ]:
# ONE HOT ENCODING DE LOS CLUSTERS - PARA EVITAR CREAR CARACTERISTICAS NUMERICAS BASADAS EN ELLOS
# Guardamos los nombres y definiciones de los clusters aparte para trabajar solo con la columna Cluster de productos
# Crear el diccionario de nombres y descripciones de clusters
cluster_dict = df_items_sp_clusters[['Cluster', 'cluster_name', 'description']].drop_duplicates().set_index('Cluster').to_dict('index')

df_items_ohe = df_items_sp_clusters.copy()

# Hacer OHE usando scikit learn
from sklearn.preprocessing import OneHotEncoder
df_items_ohe = df_items_ohe.drop(columns=['Cluster', 'description'])

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
df_items_ohe = ohe.fit_transform(df_items_ohe[['cluster_name']])
df_items_ohe = pd.DataFrame(df_items_ohe, columns=ohe.get_feature_names_out(['cluster_name']))
df_items_ohe = pd.concat([df_items_sp_clusters[['item']], df_items_ohe], axis=1)

display(df_items_ohe.head())

## Seleccion / Creacion caracteristicas

In [ ]:
# Creacion caracteristicas globales para tiendas (luego iremos filtrando)
fc_config = {
    'target_df': 'shops',  # El DataFrame que contiene información sobre las tiendas
    'agg_primitives': [
        'mean',              # Promedio
        'sum',               # Suma
        'count'              # Totales
    ],
    'trans_primitives': [
        'percentile'
    ],
    'max_depth': 2  # Profundidad máxima de las características generadas
}

feature_matrix, feature_defs = fcm.create_features(df_items_ohe, df_shops, df_ws, fc_config)

In [ ]:
feature_matrix.head()

In [ ]:
# Revision definiciones caracteristicas creadas
features_descriptions = fcm.get_features_descriptions(feature_defs)

display(type(features_descriptions))

display(features_descriptions)

## Seleccion caracteristicas

In [ ]:
substrings = substrings = [
    'items',
    'units',
    'raw_earn',
    'sell_price',
    'cluster_name'
]

feat_selected = fcm.select_features(features_descriptions, substrings)

print(f'Length Feats Selected: {len(feat_selected)}')
# display(feat_selected)

feature_matrix_selected = feature_matrix[feat_selected].copy()

# display(feature_matrix_selected.head(5))

# Aplicar filtros a las caracteristicas creadas
# Remove low information, highly correlated, highly null and single value features
filtered_feature_matrix = fcm.filter_feature_matrix(feature_matrix_selected)

# Renombrar características si es necesario # Evitamos este paso de momento
# feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, price_range_features)
feature_matrix_renamed = filtered_feature_matrix.copy()

display(feature_matrix_renamed.head(5))

## Preprocesamiento datos

In [ ]:
scaled_df = pm.preprocess_features(feature_matrix_renamed)

## Kmeans

Elbow & Silhoutte

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 9)

In [ ]:
optimal_k = 5 # codo + silueta

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

Analisis

In [ ]:
# Vemos varianza acumulada en componentes principales
variance_ratio = km_m.view_pca_variance(scaled_df)

acc_var = km_m.calculate_accumulated_variance(variance_ratio)

display(acc_var)

Importancia caracteristicas

In [ ]:
# Importancia de caracteristicas
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([scaled_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

IMPORTANCE_THRES=0.05
importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=IMPORTANCE_THRES)

# Seleccionamos solo las caracteristicas que tienen importancia por encima o igual del umbral
selected_features = importance_df[importance_df['Importance'] >= IMPORTANCE_THRES]

# Creamos una lista con las features más importantes usando la columna feature de importance_df
most_important_features = selected_features['Feature'].tolist()

# display(importance_df.head(10))

## Dim redux

In [ ]:
# Ejecutar PCA y clustering
n_components = 5 # Segun visto en varianza acumulada -> casi 95%

# Obtenemos scaled df con componentes y clusters
df_with_pca, loadings_df = km_m.pca_and_cluster(scaled_df, n_clusters=optimal_k, n_components=n_components)

# # Mostrar el DataFrame con PCA y clustering
# display(df_with_pca.head(5))

# Graficar la cantidad de items por cluster
show_clusters = km_m.plot_cluster_sizes(df_with_pca)

## Descripcion de clusters

In [ ]:
# Visualizacion clusters
cols = n_components + 1 # -> Seleccionamos solo las columnas de PC + Cluster
df_pairplot = df_with_pca.iloc[:, -(cols):]

df_pairplot['Cluster'] = df_with_pca['Cluster']
sns.pairplot(df_pairplot[0:], hue='Cluster', palette='viridis')

In [ ]:
# T SNE visualizacion de clusters
km_m.tsne_visualization(df_pairplot, n_components=2, perplexity=30)

In [ ]:
# Seleccionamos las features según la lista most_important_features obtenida con random forest
importance_df_filt = importance_df[importance_df['Feature'].isin(most_important_features)]

# Crear y mostrar descripciones de los clusters
description_df = km_m.create_cluster_descriptions(df_with_pca, importance_df_filt)
print('###########################################################')
print('Descripcion de los clusters')
print('###########################################################')
# Ordenamos columnas de description_df usando sort
description_df = description_df[sorted(description_df.columns)]
display(description_df)

In [ ]:
# Añadimos clusters a df_items
df_shops = df_shops.reset_index(drop=True) # Recuperamos indices originales

df_shops_with_clusters = df_shops.merge(df_with_pca[['Cluster']], left_index=True, right_index=True)

df_shops_with_clusters.head()

In [ ]:
# Crear dataframe con las descripciones de los clusters
cluster_summary = {
    0: {
        'cluster_name': 'Tiendas con ventas bajas y productos moderados',
        'description': 'Tiendas con precios de venta y ganancias acumuladas bajas. Suelen vender productos de rendimiento moderado, pero no destacan en ninguna métrica importante.'
    },
    1: {
        'cluster_name': 'Tiendas con ventas estables y productos promedio',
        'description': 'Tiendas que presentan una mezcla de productos con ingresos moderados. Aunque no alcanzan picos altos de ventas, mantienen un rendimiento equilibrado en la mayoría de las métricas.'
    },
    2: {
        'cluster_name': 'Tiendas con productos de alto rendimiento y consistencia',
        'description': 'Tiendas que destacan por sus productos de alto valor y ventas consistentes. Generan buenos ingresos y tienen un mix de productos que incluye tanto top performers como moderados.'
    },
    3: {
        'cluster_name': 'Tiendas con alto volumen de ventas y productos top',
        'description': 'Tiendas que muestran un rendimiento excepcional, con precios de venta altos y un gran volumen de ingresos. Venden principalmente productos de alto rendimiento con baja variabilidad.'
    },
    4: {
        'cluster_name': 'Tiendas con ventas fluctuantes y productos de baja rentabilidad',
        'description': 'Tiendas con productos de bajo valor que experimentan fluctuaciones en las ventas. Aunque presentan algunos productos con buen desempeño, en general tienen un rendimiento irregular.'
    }
}

cluster_summary_df = pd.DataFrame.from_dict(cluster_summary, orient='index')

# Asumiendo que tienes un dataframe `items_df` que tiene una columna 'cluster' con los números de cluster
# Realiza un merge con el resumen de clusters para añadir los nombres y descripciones al dataframe original
shops_with_cluster_info = df_shops_with_clusters.merge(cluster_summary_df, left_on='Cluster', right_index=True, how='left')

shops_with_cluster_info = shops_with_cluster_info.rename(columns={'Cluster':'cluster'})

# Resultado: `shops_with_cluster_info` tendrá las columnas adicionales 'cluster_name' y 'description'
display(shops_with_cluster_info.head())

In [ ]:
shops_with_cluster_info

In [ ]:
km_m.plot_clusters_with_name(shops_with_cluster_info)

## Guardar CSV

In [ ]:
# Guardamos df como csv
from google.colab import files

file_name = 'shops_products_mix.csv'

# # Exportar DataFrame a CSV (reemplaza 'df' por el nombre de tu DataFrame)
# shops_with_cluster_info.to_csv(file_name, index=False)

# # Descargar el archivo CSV
# files.download(file_name)

# **TIENDAS: SALES w EVENTS**
___
___

In [ ]:
# Recuperamos full df con eventos
df_ws = get_data('daily_sales_with_events.csv')

# Seleccionar solo años 2014 a 2016 de df_ws
df_ws = df_ws[(df_ws['year'] >= 2014) & (df_ws['year'] <= 2016)]

# Cambiamos NaN de event por None
df_ws['event'] = df_ws['event'].fillna('None')

display(df_ws.shape)

## Creacion caracteristicas

In [ ]:
# OHE para columna event en df_ws usando sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder

# Seleccionar la columna 'event' para codificar
events = df_ws[['event']]

# Crear el OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Ajustar y transformar los eventos con OHE
event_encoded = ohe.fit_transform(events)

# Crear un DataFrame con las columnas codificadas
event_encoded_df = pd.DataFrame(event_encoded, columns=ohe.get_feature_names_out(['event']))

# Concatenar con el DataFrame original de ventas
df_ws_ohe = pd.concat([df_ws.reset_index(drop=True), event_encoded_df], axis=1)



In [ ]:
display(df_ws_ohe.head())

In [ ]:
# Seteamos index de df_shops a columna store_code
# df_shops.set_index('store_code', inplace=True)
df_shops.head()

In [ ]:
import featuretools as ft

# Crear un EntitySet
es = ft.EntitySet(id='sales_data')

# Agregar entidades
es = es.add_dataframe(dataframe_name="items", dataframe=df_items, index="item")
es = es.add_dataframe(dataframe_name="shops", dataframe=df_shops, index="store_code")
es = es.add_dataframe(dataframe_name="sales", dataframe=df_ws_ohe, index="id",
                      time_index="week",)
                      # secondary_time_index={'year': ['units', 'sell_price', 'raw_earn']})

# Definir relaciones
es = es.add_relationship("items", "item", "sales", "item")
es = es.add_relationship("shops", "store_code", "sales", "store_code")

# Generar características automáticas, ahora centradas en "items"
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="shops",  # Objetivo: productos
                                      agg_primitives=[
                                          'min',               # Mínimo
                                          'max',               # Máximo
                                          'mean',              # Promedio
                                          'std',               # Desviación estándar
                                          'sum',               # Suma
                                          'count'              # Totales
                                      ],
                                      trans_primitives=[
                                          # Month no hace falta ya lo tenemos en esta iteracion
                                          # 'month',  # Extrae el mes para observar patrones estacionales a nivel semanal
                                          'is_weekend'  # Indica si la semana incluye fines de semana
                                      ],
                                      max_depth=2)

feature_matrix.head(10)

In [ ]:
# Revision definiciones caracteristicas creadas
features_descriptions = fcm.get_features_descriptions(feature_defs)

display(type(features_descriptions))

display(features_descriptions)

## Seleccion caracteristicas

In [ ]:
# V3
substrings = ['raw_earn', 'units', 'event'] # Nos quedamos solo con las que afectan a ventas y eventos

print(f'Length All features: {len(features_descriptions)}')

feat_selected = fcm.select_features(features_descriptions, substrings)

print(f'Length Feats Selected: {len(feat_selected)}')
# display(feat_selected)

feature_matrix_selected = feature_matrix[feat_selected].copy()

# display(feature_matrix_selected.head(5))

# Aplicar filtros a las caracteristicas creadas
# Remove low information, highly correlated, highly null and single value features
filtered_feature_matrix = fcm.filter_feature_matrix(feature_matrix_selected)

# Renombrar características si es necesario # Evitamos este paso de momento
# feature_matrix_renamed = fcm.rename_features(filtered_feature_matrix, price_range_features)
feature_matrix_renamed = filtered_feature_matrix.copy()

display(feature_matrix_renamed.head(5))

## Preprocesamiento de los datos

In [ ]:
scaled_df = pm.preprocess_features(feature_matrix_renamed)

## Kmeans

Elbow & Silouette

In [ ]:
km_m.plot_elbow_silhouette(scaled_df, 9) # No podemos usar más de n_shops - 1

In [ ]:
optimal_k = 6

kmeans_model, feature_matrix_with_clusters = km_m.apply_kmeans_and_plot(optimal_k, scaled_df, feature_matrix_renamed)

feature_matrix_with_clusters.head(5)

Analisis

PCAs

In [ ]:
# Vemos varianza acumulada en componentes principales
variance_ratio = km_m.view_pca_variance(scaled_df)

acc_var = km_m.calculate_accumulated_variance(variance_ratio)

display(acc_var)

Importancia caracteristicas

In [ ]:
# Importancia de caracteristicas
# Preprocesamos datos
cluster_col='Cluster'
cols_to_scale = feature_matrix_with_clusters.columns.tolist()
df_clusters = feature_matrix_with_clusters[[cluster_col]].copy().reset_index(drop=True)
cols_to_scale.remove(cluster_col) # Columna Objetivo

# Combinar los DataFrames basándose en la columna 'Index'
preprocessed_df = pd.concat([scaled_df, df_clusters], axis=1)

clusters = kmeans_model.labels_

IMPORTANCE_THRES=0.05
importance_df = km_m.get_feature_importances(preprocessed_df, clusters, imp_threshold=IMPORTANCE_THRES)

# Seleccionamos solo las caracteristicas que tienen importancia por encima o igual del umbral
selected_features = importance_df[importance_df['Importance'] >= IMPORTANCE_THRES]

# Creamos una lista con las features más importantes usando la columna feature de importance_df
most_important_features = selected_features['Feature'].tolist()

# display(importance_df.head(10))

## Dimension Redux

In [ ]:
# Ejecutar PCA y clustering
n_components = 5 # Segun visto en varianza acumulada -> casi 95%

# Obtenemos scaled df con componentes y clusters
df_with_pca, loadings_df = km_m.pca_and_cluster(scaled_df, n_clusters=optimal_k, n_components=n_components)

# # Mostrar el DataFrame con PCA y clustering
# display(df_with_pca.head(5))

# Graficar la cantidad de items por cluster
show_clusters = km_m.plot_cluster_sizes(df_with_pca)

## Descripcion de los clusters

In [ ]:
# Visualizacion clusters
cols = n_components + 1 # -> Seleccionamos solo las columnas de PC + Cluster
df_pairplot = df_with_pca.iloc[:, -(cols):]

df_pairplot['Cluster'] = df_with_pca['Cluster']
sns.pairplot(df_pairplot[0:], hue='Cluster', palette='viridis')

In [ ]:
# T SNE visualizacion de clusters
km_m.tsne_visualization(df_pairplot, n_components=2, perplexity=30)

Mostrar descripciones

In [ ]:
# Seleccionamos las features según la lista most_important_features obtenida con random forest
importance_df_filt = importance_df[importance_df['Feature'].isin(most_important_features)]

# Crear y mostrar descripciones de los clusters
description_df = km_m.create_cluster_descriptions(df_with_pca, importance_df_filt)
print('###########################################################')
print('Descripcion de los clusters')
print('###########################################################')
# Ordenamos columnas de description_df usando sort
description_df = description_df[sorted(description_df.columns)]
display(description_df)

In [ ]:
# Asignacion clusters a shops
df_shops = df_shops.reset_index(drop=True) # Recuperamos indices originales

df_shops_with_clusters = df_shops.merge(df_with_pca[['Cluster']], left_index=True, right_index=True)

df_shops_with_clusters.head()

In [ ]:
# Crear dataframe con las descripciones de los clusters
cluster_summary = {
    0: {
        'cluster_name': 'Tiendas con impacto negativo de eventos',
        'description': 'Tiendas cuyas ventas y unidades muestran un impacto negativo significativo durante la mayoría de los eventos, con bajas ventas en eventos como el Año Nuevo, Super Bowl y Pascua.'
    },
    1: {
        'cluster_name': 'Tiendas con rendimiento moderado en eventos',
        'description': 'Tiendas que tienen un rendimiento promedio durante eventos importantes. Aunque no tienen grandes picos, muestran una ligera mejora en ventas y unidades durante eventos como el Super Bowl y Pascua.'
    },
    2: {
        'cluster_name': 'Tiendas con alta sensibilidad a eventos',
        'description': 'Tiendas que experimentan un rendimiento excepcionalmente alto en ventas durante eventos importantes. Estas tiendas son altamente dependientes de eventos como el Año Nuevo y el Super Bowl para maximizar sus ingresos y ventas.'
    },
    3: {
        'cluster_name': 'Tiendas con rendimiento negativo pero estable',
        'description': 'Tiendas que muestran un rendimiento negativo moderado durante eventos, pero con poca variabilidad en sus métricas. Su desempeño no mejora significativamente con los eventos.'
    },
    4: {
        'cluster_name': 'Tiendas con bajo impacto en eventos',
        'description': 'Tiendas con un impacto marginalmente negativo en algunos eventos, pero sin grandes fluctuaciones en su rendimiento general. Mantienen un comportamiento relativamente estable durante eventos como el Año Nuevo y el Super Bowl.'
    },
    5: {
        'cluster_name': 'Tiendas con fluctuaciones estacionales',
        'description': 'Tiendas que presentan una ligera variabilidad en ventas durante algunos eventos, con un desempeño moderado en ingresos y unidades. Su rendimiento puede mejorar levemente durante fechas clave como el Super Bowl.'
    }
}


cluster_summary_df = pd.DataFrame.from_dict(cluster_summary, orient='index')

# Realiza un merge con el resumen de clusters para añadir los nombres y descripciones al dataframe original
shops_with_cluster_info = df_shops_with_clusters.merge(cluster_summary_df, left_on='Cluster', right_index=True, how='left')

# Resultado: `items_with_cluster_info` tendrá las columnas adicionales 'cluster_name' y 'description'
display(shops_with_cluster_info.head())

In [ ]:
km_m.plot_clusters_with_name(shops_with_cluster_info)

## Guardamos csv

In [ ]:
# Guardamos df como csv
from google.colab import files

file_name = 'shops_clusters_events_influence_sales.csv'

# # Exportar DataFrame a CSV (reemplaza 'df' por el nombre de tu DataFrame)
# shops_with_cluster_info.to_csv(file_name, index=False)

# # Descargar el archivo CSV
# files.download(file_name)